In [1]:
from langchain_ollama import OllamaLLM
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from db_create import CargaDeArchivos
import re

In [2]:
models= ['duckdb-nsql:latest','vicuna:latest','sqlcoder:latest','codeqwen:latest','deepseek-coder:6.7b']

In [3]:
def fetch_schema(conn):
    """
    Fetch the schema from a DuckDB database, including column descriptions.

    Args:
        conn: Active DuckDB connection.

    Returns:
        dict: A dictionary where keys are table names and values are dictionaries 
              of column names, data types, and descriptions optimized for SQL agent queries.
    """
    cursor = conn.cursor()

    # Get all table names
    cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'")
    tables = [row[0] for row in cursor.fetchall()]

    schema_info = {}

    # Predefined column descriptions optimized for SQL agent interpretation
    column_descriptions = {
        "cases": {
            "id": "Primary key. Unique identifier for each case.",
            "insurance": "Foreign key. Unique identifier for the related insurance policy.",
            "avg_time": "Time duration (in seconds) from case initiation to closure.",
            "type": "Category of insurance policy (e.g., health, auto, home).",
            "branch": "Branch where the policy was issued.",
            "ramo": "Specific coverage type under the insurance policy.",
            "brocker": "Broker responsible for selling the insurance policy.",
            "client": "Client who purchased the insurance policy.",
            "creator": "Employee responsible for managing the case.",
            "value": "Monetary value of the insurance policy.",
            "approved": "Approval status: 1 = Approved, 0 = Not Approved.",
            "insurance_creation": "Timestamp of when the insurance policy was created.",
            "insurance_start": "Timestamp of when the policy coverage begins.",
            "insurance_end": "Timestamp of when the policy coverage expires.",
        },
        "activity": {
            "id": "Primary key. Unique identifier for each recorded activity.",
            "case": "Foreign key. Identifier of the case this activity belongs to.",
            "timestamp": "Timestamp indicating when the activity took place.",
            "name": "Descriptive name of the activity performed.",
            "case_index": "Alias for 'id'. Unique identifier for the activity record.",
            "tpt": "Time duration (in seconds) for this specific activity.",
        }
    }

    # Fetch column information for each table
    for table in tables:
        cursor.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = '{table}'
        """)
        columns = cursor.fetchall()

        schema_info[table] = {
            col[0]: {
                "type": col[1],
                "description": column_descriptions.get(table, {}).get(col[0], "Relevant column for querying this table.")
            }
            for col in columns
        }

    return schema_info

In [4]:
a= CargaDeArchivos()
a.run_carga()
db_conn= a.conn
schema=fetch_schema(db_conn)

In [5]:
class State(TypedDict):
    """
    Represents the state of the workflow, including the question, schema, database connection,
    relevance, SQL query, query result, and other metadata.
    """
    question: str
    schema: str
    db_conn: None
    relevance: str
    sql_query: str
    query_result: str
    sql_error: bool
    final_answer: str
    attempts: int

In [12]:
def convert_nl_to_sql(state: State):
    """
    Converts a natural language question into an SQL query based on the database schema.

    Args:
        state (State): The current state of the workflow.
        config (RunnableConfig): Configuration for the runnable.

    Returns:
        State: Updated state with the generated SQL query.
    """
    question = state["question"]
    schema= state["schema"]
    print(f"Converting question to SQL {question}")
    system = """
    You are an SQL assistant specialized in DuckDB. Your task is to generate accurate SQL queries based on natural language questions, following the provided schema.

    ### Database Schema  
    #### Table: "cases"
    - "id" (VARCHAR): Primary key.
    - "insurance" (BIGINT): Foreign key to insurance.
    - "avg_time" (DOUBLE): Duration (seconds) from case initiation to closure.
    - "type" (VARCHAR): Insurance category.
    - "branch" (VARCHAR): Policy branch.
    - "ramo" (VARCHAR): Coverage type.
    - "broker" (VARCHAR): Broker for the policy.
    - "state" (VARCHAR): Current case state.
    - "client" (VARCHAR): Client who bought the insurance.
    - "creator" (VARCHAR): Employee managing the case.
    - "value" (BIGINT): Insurance monetary value.
    - "approved" (BOOLEAN): TRUE if approved, else FALSE.
    - "insurance_creation" (TIMESTAMP_NS): Policy creation timestamp.
    - "insurance_start" (TIMESTAMP_NS): Coverage start timestamp.
    - "insurance_end" (TIMESTAMP_NS): Coverage end timestamp.

    #### Table: "activity"
    - "id" (BIGINT): Primary key.
    - "case" (VARCHAR): Foreign key to "cases"."id".
    - "timestamp" (TIMESTAMP_NS): Activity timestamp.
    - "name" (VARCHAR): Name of the activity.
    - "case_index" (BIGINT): Alias for "id".
    - "tpt" (DOUBLE): Activity duration (seconds).

    ### Query Guidelines  
    1. Use Table Aliases: "cases" → c, "activity" → a.
    2. Always Reference Columns with Aliases: c."id", a."case".
    3. Handle Aggregations: Include non-aggregated columns in GROUP BY.
    4. Date & Time Calculations: Use EXTRACT(DAY FROM ...) for durations.
    5. Filtering Conditions: Use TRUE/FALSE for boolean values.
    6. Use Explicit Joins: Avoid implicit joins.
    7. Optimize for Performance: Use indexes, avoid unnecessary calculations, and limit results when needed.
    8. Restrict Queries to Existing Tables: Only use "cases" and "activity" tables.
    
    ### Output Format  
    Respond with only the SQL query, no explanations or additional text.
    """


    llm= OllamaLLM(model="duckdb-nsql:latest",temperature="0.0")
    
    convert_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", "Question: {question}"),
        ]
    )
    sql_generator = convert_prompt | llm
    result = sql_generator.invoke({"question": question,"schema":schema})
    message= re.sub(r'^\s*```sql\s*|\s*```$', '', result.strip(), flags=re.IGNORECASE)
    state["sql_query"] = message
    print(f"Generated SQL query: {state['sql_query']}")
    return state


def execute_sql(state:State):
    """
    Executes the SQL query on the  database and retrieves the results.

    Args:
        state (State): The current state of the workflow.
        config (RunnableConfig): Configuration for the runnable.

    Returns:
        State: Updated state with the query results or error information.
    """
    sql_query = state["sql_query"].strip()
    db_conn = state["db_conn"]  
    print(f"Executing SQL query: {sql_query}")

    try:
        # Ensure the query targets only the allowed tables
        allowed_tables = ["cases", "activity"]
        if not any(table in sql_query.lower() for table in allowed_tables):
            raise ValueError(f"Query must target only the tables: {', '.join(allowed_tables)}.")

        # Execute the SQL query using the connection
        cursor = db_conn.cursor()
        cursor.execute(sql_query)

        # Fetch results if it's a SELECT query
        if sql_query.lower().startswith("select"):
            rows = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]

            # Format the output
            if rows:
                formatted_result = "\n".join(
                    ", ".join(f"{col}: {row[idx]}" for idx, col in enumerate(columns))
                    for row in rows
                )
                print("SQL SELECT query executed successfully.")
            else:
                formatted_result = "No results found."
                print("SQL SELECT query executed successfully but returned no rows.")

            state["query_rows"] = rows
        else:
            formatted_result = "The action has been successfully completed."
            print("SQL command executed successfully.")

        state["query_result"] = formatted_result
        state["sql_error"] = False

    except Exception as e:
        state["query_result"] = f"Error executing SQL query: {str(e)}"
        state["sql_error"] = True
        print(f"Error executing SQL query: {str(e)}")
    print(state['query_result'])
    return state

In [13]:
test_cases = [
    {"question": "How many cases are there in total in the database?",
    "expected_sql": 'SELECT COUNT(*) FROM cases;'},
    {"question": "How many activities have been recorded?",
    "expected_sql": 'SELECT COUNT(*) FROM activity;'},
    {"question": "What are the different types of insurance available?",
    "expected_sql": 'SELECT DISTINCT type FROM cases;'},
    {"question": "What is the total value of approved insurance policies?",
    "expected_sql": 'SELECT SUM(value) FROM cases WHERE approved = 1;'},
    {"question": "How many cases were created in January?",
    "expected_sql": 'SELECT COUNT(*) FROM cases WHERE EXTRACT(MONTH FROM insurance_creation) = 1;'},
    {"question": "Who is the broker with the most assigned cases?",
    "expected_sql": 'SELECT brocker FROM cases GROUP BY brocker ORDER BY COUNT(*) DESC LIMIT 1;'},
    {"question": "What is the most frequent activity in the database?",
    "expected_sql": 'SELECT name FROM activity GROUP BY name ORDER BY COUNT(*) DESC LIMIT 1;'},
    {"question": "What is the total insurance value for each type of 'ramo'?",
    "expected_sql": 'SELECT ramo, SUM(value) FROM cases GROUP BY ramo;'},
    {"question": "On average, how long does it take for an insurance policy to be approved?",
    "expected_sql": 'SELECT AVG(EXTRACT(DAY FROM insurance_start - insurance_creation)) FROM cases WHERE approved = 1;'},
    {"question": "What is the average number of activities per case?",
    "expected_sql": 'SELECT COUNT(activity.id) / COUNT(DISTINCT cases.id) FROM activity INNER JOIN cases ON activity.case = cases.id;'},
    {"question": "What is the most frequent activity performed in approved cases?",
    "expected_sql": 'SELECT activity.name FROM activity INNER JOIN cases ON activity.case = cases.id WHERE cases.approved = TRUE GROUP BY activity.name ORDER BY COUNT(activity.id) DESC LIMIT 1;'},
    {"question": "What is the total duration of all activities for each case?",
    "expected_sql": 'SELECT activity.case, SUM(activity.tpt) FROM activity GROUP BY activity.case;'},
    {"question": "What is the total value of cases that have at least one recorded activity?",
    "expected_sql": 'SELECT SUM(cases.value) FROM cases WHERE cases.id IN (SELECT DISTINCT case FROM activity);'}
]

In [14]:
import time
import pandas as pd

def time_query_generation(state):
    """
    Measures the execution time of the model generating an SQL query.

    Args:
        state (dict): The state object that stores the query and result.

    Returns:
        dict: Updated state containing the query, execution time, and SQL output.
    """
    start_time = time.time()
    try:
        convert_nl_to_sql(state)  # Generates SQL query
        execute_sql(state)  # Executes SQL query
    except Exception as e:
        state["sql_query"] = f"Error: {e}"

    execution_time = time.time() - start_time
    state["execution_time"] = execution_time
    return state

In [15]:
results = []

# Run tests
for test in test_cases:
    state = {"question": test["question"], "expected_sql": test["expected_sql"],"schema":schema,"db_conn":db_conn}
    result_state = time_query_generation(state)
    
    results.append({
        "Question": test["question"],
        "Expected SQL Query": test["expected_sql"],
        "Generated SQL Query": result_state['sql_query'],
        "Query Result": result_state["query_result"],
        "Execution Time (s)": result_state["execution_time"]
    })

# Convert results to DataFrame
df_results = pd.DataFrame(results)

Converting question to SQL How many cases are there in total in the database?
Generated SQL query: SELECT COUNT(*) FROM cases;
Executing SQL query: SELECT COUNT(*) FROM cases;
SQL SELECT query executed successfully.
count_star(): 1000
Converting question to SQL How many activities have been recorded?
Generated SQL query: SELECT COUNT(*) FROM activity
Executing SQL query: SELECT COUNT(*) FROM activity
SQL SELECT query executed successfully.
count_star(): 23233
Converting question to SQL What are the different types of insurance available?
Generated SQL query: SELECT DISTINCT type FROM cases;
Executing SQL query: SELECT DISTINCT type FROM cases;
SQL SELECT query executed successfully.
type: Issuance
type: Policy onboarding
type: Renewal
Converting question to SQL What is the total value of approved insurance policies?
Generated SQL query: SELECT SUM(value) FROM cases WHERE approved = TRUE;
Executing SQL query: SELECT SUM(value) FROM cases WHERE approved = TRUE;
SQL SELECT query executed 

In [16]:
df_results

,Question,Expected SQL Query,Generated SQL Query,Query Result,Execution Time (s)
0,How many cases are there in total in the datab...,SELECT COUNT(*) FROM cases;,SELECT COUNT(*) FROM cases;,count_star(): 1000,1.036623
1,How many activities have been recorded?,SELECT COUNT(*) FROM activity;,SELECT COUNT(*) FROM activity,count_star(): 23233,0.934192
2,What are the different types of insurance avai...,SELECT DISTINCT type FROM cases;,SELECT DISTINCT type FROM cases;,type: Issuance\ntype: Policy onboarding\ntype:...,1.065332
3,What is the total value of approved insurance ...,SELECT SUM(value) FROM cases WHERE approved = 1;,SELECT SUM(value) FROM cases WHERE approved = ...,"sum(""value""): 3751100",1.303513
4,How many cases were created in January?,SELECT COUNT(*) FROM cases WHERE EXTRACT(MONTH...,SELECT COUNT(*) FROM cases WHERE EXTRACT(MONTH...,count_star(): 355,1.791762
5,Who is the broker with the most assigned cases?,SELECT brocker FROM cases GROUP BY brocker ORD...,"SELECT broker, COUNT(*) as case_count\nFROM ca...","broker: Stanley, case_count: 9",2.131488
6,What is the most frequent activity in the data...,SELECT name FROM activity GROUP BY name ORDER ...,"SELECT a.name, COUNT(*) \nFROM activity a \nJO...","name: Enviar a Revisión suscripción, count_sta...",2.688401
7,What is the total insurance value for each typ...,"SELECT ramo, SUM(value) FROM cases GROUP BY ramo;","SELECT ramo, SUM(value) FROM cases GROUP BY ramo;","ramo: Auto Insurance, sum(""value""): 338600\nra...",1.393619
8,"On average, how long does it take for an insur...",SELECT AVG(EXTRACT(DAY FROM insurance_start - ...,SELECT AVG(approved) FROM cases,Error executing SQL query: Binder Error: No fu...,1.156006
9,What is the average number of activities per c...,SELECT COUNT(activity.id) / COUNT(DISTINCT cas...,SELECT AVG(a.case_index) FROM activity a JOIN ...,avg(a.case_index): 5065.7115310119225,1.840317


In [11]:
#df_results.to_csv("results_duckdb-nsql.csv", index=False)